In [1]:
import sys, importlib as impL
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/vae_torch')
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery')
import vae_torch_model as vtm
import vae_torch as vt
#from data_classes import khs_dataset_v2
import vae_scripts as vs
import sae_torch as st
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import vae_utils as vu
import pandas as pd
import helperFuncs as funcH

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option("display.precision", 3)
pd.set_option('display.max_rows', None)

/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
def get_acc_err(experiments_folder, exp_base_name,cf_int,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=1):
    ae_fold_name = os.path.join(experiments_folder, exp_base_name + str(cf_int).zfill(z_fill_int))
    ae_f_name = os.path.join(ae_fold_name, ae_f_name_base)
    vfz = np.load(ae_f_name, allow_pickle=True)
    loss_log_dict = {}
    n = 0
    loss_log_dict[data_log_key] = vfz.item().get(data_log_key)
    if loss_log_dict[data_log_key] is None:
        print("cf("+str(cf_int)+") --> loss_log_dict"+str(data_log_key)+"] is none")
        return None, None, None
    n = len(loss_log_dict[data_log_key])
    if verbose>0:
        print(str(cf_int), ', ', data_log_key, " - log is loaded with len: ", n)
    if loss_key not in loss_log_dict[data_log_key][0].keys():
        return n, None, None

    if loss_key in loss_log_dict[data_log_key][0]:
        vec_len = len(loss_log_dict[data_log_key])
        los_vec_cur = [loss_log_dict[data_log_key][l][loss_key] for l in range(0, vec_len)]
        label_str = str(cf_int) + '_' + data_log_key + '_' + loss_key
        if verbose>0:
            print(label_str, los_vec_cur[-3:], "\nmax({:4.2f}),min({:4.2f})".format(np.max(los_vec_cur), np.min(los_vec_cur)))
        return n, np.min(los_vec_cur), np.max(los_vec_cur)
    return None, None, None

In [3]:
impL.reload(vu)
experiments_folder='/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/experiments/FM/'
config_folder = '/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/configs/'
z_fill_int=2
loss_key='reconstruction' # sparsity, bottleneck_kmeans, bottleneck_act
data_log_keys=['tr_te']  # ['tr_va', 'va', 'te']
exp_base_name='exp_conv_ae_simple_is28_cf'
ae_f_name_base='ae_ft_conv_ae_simple_is28.npy'
cf_int_arr=[201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218]
#cf_int_arr=[201,204,205,207,213,214]
#cf_int_arr=[201,204,205,216,217,218]
df1 = pd.DataFrame(index=cf_int_arr, columns=['spMethod','spW','spR','kl_div','bact','lact','rcErr','lr','rcRed'])
df2 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
df3 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
for i in range(0,len(cf_int_arr)):
    cf = cf_int_arr[i]
    #load model yaml as config
    config_file = os.path.join(config_folder, 'conf_autoencoder_'+str(cf)+'.yaml')
    CONF_PARAMS_ = funcH.CustomConfigParser(config_file=config_file)
    df1.iloc[i]['spW'] = float(CONF_PARAMS_.MODEL.SPARSITY_WEIGHT)
    if df1.iloc[i]['spW'] > 0:
        df1.iloc[i]['spMethod'] = CONF_PARAMS_.MODEL.SPARSITY_ERROR
        df1.iloc[i]['spR'] = CONF_PARAMS_.MODEL.SPARSITY_REDUCTION
    else:
        df1.iloc[i]['spW'] = None
    df1.iloc[i]['kl_div'] = CONF_PARAMS_.MODEL.KL_DIV_RHO if 'kl_' in str(CONF_PARAMS_.MODEL.SPARSITY_ERROR) else None
    try:
        df1.iloc[i]['bact'] = (CONF_PARAMS_.MODEL.LAYERS.encoder.l04_act).replace('type: ','').replace(',dim:','')
    except:
        pass
    try:
        df1.iloc[i]['lact'] = CONF_PARAMS_.MODEL.LAYERS.decoder.l01_act.replace('type: ','')
    except:
        pass
    df1.iloc[i]['rcErr'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_FUNCTION
    df1.iloc[i]['lr'] = CONF_PARAMS_.MODEL.LEARNING_RATE
    df1.iloc[i]['rcRed'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_REDUCTION
    
    
    data_log_key = 'tr_te'
    loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
    n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['n'] = n

    df2.iloc[i]['rErrMin'] = mn
    df2.iloc[i]['rErrMax'] = mx
    
    loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['kmMin'] = mn
    df2.iloc[i]['kmMax'] = mx
    
    loss_key = 'bottleneck_act' #  sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['bActMin'] = mn
    df2.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'

    loss_key = 'sparsity' #  
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df2.iloc[i]['bErrMin'] = mn
    df2.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'

    
    data_log_key = 'te'
    loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
    n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['n'] = n

    df3.iloc[i]['rErrMin'] = mn
    df3.iloc[i]['rErrMax'] = mx
    
    loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['kmMin'] = mn
    df3.iloc[i]['kmMax'] = mx
    
    loss_key = 'bottleneck_act' #  sparsity
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['bActMin'] = mn
    df3.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'

    loss_key = 'sparsity' #  
    _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
    df3.iloc[i]['bErrMin'] = mn
    df3.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'

In [4]:
pd.set_option("display.precision", 2)
print('experiments explanations')
print(df1)

experiments explanations
          spMethod    spW        spR  kl_div      bact  lact rcErr     lr rcRed
201  kl_divergence   0.01  batchmean  0.0001  Softmax1   NaN   MSE  0.001   sum
202        l2_norm   0.01  batchmean    None  Softmax1   NaN   MSE  0.001  mean
203        l2_norm  0.001        sum    None  Softmax1   NaN   MSE  0.001  mean
204  kl_divergence    0.5  batchmean  0.0001  Softmax1   NaN   MSE  0.001   sum
205  kl_divergence      1  batchmean  0.0001  Softmax1   NaN   MSE  0.001   sum
206            NaN   None        NaN    None  Softmax1   NaN   MSE  0.001  mean
207            NaN   None        NaN    None  Softmax1   NaN   MSE  0.001   sum
208        l1_norm   0.01  batchmean    None  Softmax1   NaN   MSE  0.001  mean
209        l1_norm  0.001        sum    None  Softmax1   NaN   MSE  0.001  mean
210        l1_norm   0.01        sum    None  Softmax1  ReLu   MSE  0.001   sum
211        l1_norm    0.1        sum    None  Softmax1   NaN   MSE  0.001   sum
212        l1_n

In [5]:
print('tr_te dataset results')
pd.set_option("display.precision", 5)
print(df2)

tr_te dataset results
       n bActMin bActMax     bErrMin     bErrMax   kmMin   kmMax     rErrMax     rErrMin
201  250  16.697  46.887      6.9583      6.9583  46.287  62.152  1.8807e+06  8.9107e+05
202   40      10   _n*c_     -289.52     -267.11  11.765  22.537      56.502      55.369
203   40      10   _n*c_     -4978.6     -4906.7  13.178  21.042      57.371      55.642
204  250  19.207  42.842      6.9583      6.9583  44.825  61.883  1.8229e+06  8.8944e+05
205  250  28.107  45.532      6.9583      6.9583  48.023  64.568  1.6562e+06   8.888e+05
206   40  8.0833   16.98           0       _n*c_   10.63   22.81      46.839      43.638
207  250  15.142   45.24           0       _n*c_  43.335   62.11  2.4229e+06  8.9924e+05
208   40   9.485  16.185       -3750       -3750  11.773  23.595      46.368      43.822
209   40      10   _n*c_      -60000      -60000      10      10      60.167      57.468
210   30  16.877  35.882      -60000      -60000  46.003  57.813  7.3034e+06  7.0631e+06

In [6]:
print('te dataset results')
print(df3)

te dataset results
       n bActMin bActMax bErrMin bErrMax  kmMin  kmMax     rErrMax     rErrMin
201  250   18.99   45.96  1.1721  1.1721  47.82  60.25  3.1283e+05  1.4921e+05
202   40      10   _n*c_ -48.426 -44.676  13.06  24.84      9.4758      9.2852
203   40      10   _n*c_ -832.51 -820.54  10.58  21.72      9.6214      9.3313
204  250   19.77      43  1.1721  1.1721  45.65  61.96   3.029e+05  1.4916e+05
205  250   25.17   43.88  1.1721  1.1721  46.55  64.12  2.7542e+05  1.4902e+05
206   40    9.89   17.06       0   _n*c_  11.98  21.89      7.8565      7.3202
207  250   15.52   45.08       0   _n*c_  41.79  63.85   4.027e+05  1.5055e+05
208   40    3.83   16.52    -625    -625  11.41  21.79      7.7772      7.3505
209   40      10   _n*c_  -10000  -10000     10     10      10.092      9.6372
210   30   15.37   36.31  -10000  -10000  46.28  59.06   1.212e+06  1.1726e+06
211   10   23.37    28.4  -10000  -10000  53.17  62.93  2.8019e+05  1.7751e+05
212   30   16.45   37.18  -10000 